In [122]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

# Configura la ruta del controlador de Chrome
chrome_driver_path = r'C:\chrome_driver\chromedriver-win32\chromedriver.exe'
driver = webdriver.Chrome(service=Service(chrome_driver_path))

In [123]:
url = 'https://www.lima-airport.com/pasajeros/vuelos'
driver.get(url)

time.sleep(3)

button2 = driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div[2]/header/div/div[2]/div/div/div/div/div/div[2]/div/div[2]/label')
button2.click()

time.sleep(3)

In [124]:
'''
def obtener_vuelos(driver):
    # Seleccionar los elementos de vuelos usando selectores CSS
    vuelos = driver.find_elements(By.CLASS_NAME, 'flightDetailsMain')

    # Lista para almacenar la información de los vuelos
    vuelos_info = []

    # Iterar sobre los elementos de vuelos encontrados
    for vuelo in vuelos:
        vuelos_info.append(vuelo.text.strip().split('\n'))  # Almacena el texto limpio del elemento

    # Crear un DataFrame, ignorando el primer registro
    
    df_vuelos = pd.DataFrame(vuelos_info[1:], columns=['Estado', 'Fecha del Vuelo', 'Hora de Salida', 'Nueva Hora de Salida', 'Destino', 'Vuelo', 'Gate'])  # Ignora el primer registro

    df_cleaned = df_vuelos.dropna(how='all')

    # Eliminar filas donde 'Estado' es None o vacío
    df_cleaned = df_cleaned[df_cleaned['Estado'].notna() & (df_cleaned['Estado'].str.strip() != '')]

    df_cleaned.reset_index(drop=True, inplace=True)

    # Mostrar el DataFrame resultante
    df_cleaned

    return df_cleaned
'''

"\ndef obtener_vuelos(driver):\n    # Seleccionar los elementos de vuelos usando selectores CSS\n    vuelos = driver.find_elements(By.CLASS_NAME, 'flightDetailsMain')\n\n    # Lista para almacenar la información de los vuelos\n    vuelos_info = []\n\n    # Iterar sobre los elementos de vuelos encontrados\n    for vuelo in vuelos:\n        vuelos_info.append(vuelo.text.strip().split('\n'))  # Almacena el texto limpio del elemento\n\n    # Crear un DataFrame, ignorando el primer registro\n    \n    df_vuelos = pd.DataFrame(vuelos_info[1:], columns=['Estado', 'Fecha del Vuelo', 'Hora de Salida', 'Nueva Hora de Salida', 'Destino', 'Vuelo', 'Gate'])  # Ignora el primer registro\n\n    df_cleaned = df_vuelos.dropna(how='all')\n\n    # Eliminar filas donde 'Estado' es None o vacío\n    df_cleaned = df_cleaned[df_cleaned['Estado'].notna() & (df_cleaned['Estado'].str.strip() != '')]\n\n    df_cleaned.reset_index(drop=True, inplace=True)\n\n    # Mostrar el DataFrame resultante\n    df_cleaned\n

In [125]:
def obtener_vuelos(driver):
    # Seleccionar los elementos de vuelos usando selectores CSS
    vuelos = driver.find_elements(By.CLASS_NAME, 'flightDetailsMain')

    # Lista para almacenar la información de los vuelos
    vuelos_info = []

    # Iterar sobre los elementos de vuelos encontrados
    for vuelo in vuelos:
        datos_vuelo = vuelo.text.strip().split('\n')
        
        # Comprobar si 'Gate' está vacío y asignar "-" si es necesario
        if len(datos_vuelo) < 7:
            datos_vuelo.append('-')  # Asignar "-" si no hay información de 'Gate'
        
        vuelos_info.append(datos_vuelo)

    # Crear un DataFrame, ignorando el primer registro
    df_vuelos = pd.DataFrame(vuelos_info[1:], columns=['Estado', 'Fecha del Vuelo', 'Hora de Salida', 'Nueva Hora de Salida', 'Destino', 'Vuelo', 'Gate'])  # Ignora el primer registro

    df_cleaned = df_vuelos.dropna(how='all')

    # Eliminar filas donde 'Estado' es None o vacío
    df_cleaned = df_cleaned[df_cleaned['Estado'].notna() & (df_cleaned['Estado'].str.strip() != '')]

    df_cleaned.reset_index(drop=True, inplace=True)

    # Mostrar el DataFrame resultante
    return df_cleaned


In [126]:
# Encuentra todos los labels con la clase 'flight-filter'
labels = driver.find_elements(By.CSS_SELECTOR, 'label.flight-filter')
df_total = pd.DataFrame()
# Recorre cada label y haz clic en ellos
for label in labels:
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(3)
    label.click()
    df_completo = pd.DataFrame()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    # Encuentra el elemento del paginador
    paginas = driver.find_element(By.XPATH, '//*[@class="pagination"]')
    elementos_li = paginas.find_elements(By.TAG_NAME, 'li')
    #driver.execute_script(f"window.scrollTo(0, {paginas_position - 500});") 
    button3 = driver.find_element(By.XPATH, '//*[@id="vuelos-contenido"]/div/div[2]/nav/ul/li[2]/a')
    button3.click()

    # Itera sobre cada elemento de la lista (excluyendo elementos de "Previous" y "Next")
    for i in range(2, 8):
        time.sleep(3)
        #enlace = li.find_element(By.TAG_NAME, 'a')
        #texto = enlace.text.strip()  # Obtiene el texto del enlace
        #if texto.isdigit():  # Verifica si es un número de página
            # Espera a que la nueva página cargue
        #time.sleep(3)  # Ajusta el tiempo según sea necesario
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Llama a la función para obtener los vuelos y apendea al DataFrame completo
        df_vuelos = obtener_vuelos(driver)
        df_completo = pd.concat([df_completo, df_vuelos], ignore_index=True)

        time.sleep(5)
        paginas_position = paginas.location['y']

        # Desplázate hasta esa posición
        driver.execute_script(f"window.scrollTo(0, {paginas_position - 500});") 
        
        time.sleep(3)
        #button3 = driver.find_element(By.XPATH,'//*[@id="vuelos-contenido"]/div/div[2]/nav/ul/li[2]/a')
        #button3.click()

        #for i in range(2, 7):  # Desde li[2] hasta li[6] (5 clics)
        button3 = driver.find_element(By.XPATH, f'//*[@id="vuelos-contenido"]/div/div[2]/nav/ul/li[{i}]/a')
        button3.click()

        print(f'Descarga enlace: {label.text}')
    # Aquí puedes agregar cualquier código que desees ejecutar después de hacer clic
    df_total = pd.concat([df_total, df_completo], ignore_index=True)
    time.sleep(1)  # Espera un segundo entre clics, si es necesario

Descarga enlace: Ayer
Descarga enlace: Ayer
Descarga enlace: Ayer
Descarga enlace: Ayer
Descarga enlace: Ayer
Descarga enlace: Ayer
Descarga enlace: Hoy
Descarga enlace: Hoy
Descarga enlace: Hoy
Descarga enlace: Hoy
Descarga enlace: Hoy
Descarga enlace: Hoy
Descarga enlace: Mañana
Descarga enlace: Mañana
Descarga enlace: Mañana
Descarga enlace: Mañana
Descarga enlace: Mañana
Descarga enlace: Mañana


In [127]:
df_total

,Estado,Fecha del Vuelo,Hora de Salida,Nueva Hora de Salida,Destino,Vuelo,Gate
0,EMBARQUE FINALIZADO,16/10/2024,00:05,00:05,NUEVA YORK,LA 2468,21
1,EMBARQUE FINALIZADO,16/10/2024,00:05,00:05,BOGOTA,LA 2444,15
2,EMBARQUE FINALIZADO,16/10/2024,00:15,00:15,SAO PAULO,LA 8005,24
3,EMBARQUE FINALIZADO,16/10/2024,00:15,00:15,MEXICO,AM 019,23
4,EMBARQUE FINALIZADO,16/10/2024,00:15,00:15,MIAMI,LA 2480,19
...,...,...,...,...,...,...,...
318,PROGRAMADO,18/10/2024,23:40,23:40,SALTA,LA 2437,-
319,PROGRAMADO,18/10/2024,23:45,23:45,MIAMI,AA 988,-
320,PROGRAMADO,18/10/2024,23:50,23:50,MONTEVIDEO,LA 2420,-
321,PROGRAMADO,18/10/2024,23:50,23:50,BUENOS AIRES,LA 1437,-


In [128]:
import pandas as pd
from datetime import datetime

# Obtener la fecha actual en el formato deseado
fecha_hoy = datetime.now().strftime('%Y-%m-%d')

# Generar el nombre del archivo con la fecha
output_file = f'data/Descarga_LAP_{fecha_hoy}.xlsx'  # Puedes incluir un número si deseas

# Exportar el DataFrame a un archivo Excel
df_total.to_excel(output_file, index=False)

In [129]:
# Guardar el resultado en un archivo CSV delimitado por pipe
df_total.to_csv(f'output/resultados_vuelos_{fecha_hoy}.csv', sep='|', index=False, header=True)
df_total.to_csv(f'carga/resultados_vuelos.csv', sep='|', index=False, header=True)

In [130]:
driver.quit()

In [131]:
import pandas as pd
import os

def consolidar_csv(directorio_entrada, ruta_salida):
    # Lista para almacenar DataFrames
    dfs = []

    # Iterar sobre todos los archivos en el directorio
    for archivo in os.listdir(directorio_entrada):
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(directorio_entrada, archivo)
            df = pd.read_csv(ruta_archivo)
            dfs.append(df)

    # Concatenar todos los DataFrames en uno solo
    df_consolidado = pd.concat(dfs, ignore_index=True)

    # Eliminar duplicados
    df_consolidado = df_consolidado.drop_duplicates()

    # Guardar el DataFrame consolidado en un nuevo archivo CSV
    df_consolidado.to_csv(ruta_salida, index=False)

# Ejemplo de uso
directorio_entrada = 'output/'
ruta_salida = 'resultado_consolidado.csv'
consolidar_csv(directorio_entrada, ruta_salida)


In [132]:
df_final

,Estado|Fecha del Vuelo|Hora de Salida|Nueva Hora de Salida|Destino|Vuelo|Gate
0,EMBARQUE FINALIZADO|09/10/2024|00:05|00:05|BOG...
1,EMBARQUE FINALIZADO|09/10/2024|00:05|00:05|NUE...
2,EMBARQUE FINALIZADO|09/10/2024|00:15|00:15|SAO...
3,EMBARQUE FINALIZADO|09/10/2024|00:15|00:15|SAO...
4,EMBARQUE FINALIZADO|09/10/2024|00:15|00:15|MIA...
...,...
1233,PROGRAMADO|17/10/2024|23:20|23:20|RIO DE JANEI...
1234,PROGRAMADO|17/10/2024|23:30|23:30|SANTIAGO|JA ...
1235,PROGRAMADO|17/10/2024|23:45|23:45|MIAMI|AA 988|-
1236,PROGRAMADO|17/10/2024|23:50|23:50|BUENOS AIRES...


In [133]:
# Guardar el resultado en un archivo CSV delimitado por pipe
df_final.to_csv(f'Consolidado_{fecha_hoy}.csv', index=False, header=True)

In [134]:
df_final.to_parquet('Consolidado.parquet', engine='pyarrow')

In [135]:
import shutil
import os

# Definir las rutas de origen y destino
ruta_origen = r'D:\DEVELOPMENT\Positron_scripts\Consolidado.parquet'
ruta_destino = r'D:\DEVELOPMENT\django_apps\DashSimMov\DashSim\Data\Consolidado.parquet'

# Verificar si el archivo de origen existe
if os.path.exists(ruta_origen):
    try:
        # Copiar el archivo
        shutil.copy(ruta_origen, ruta_destino)
        print(f'Archivo copiado de {ruta_origen} a {ruta_destino}.')
    except Exception as e:
        print(f'Ocurrió un error al copiar el archivo: {e}')
else:
    print(f'El archivo {ruta_origen} no existe.')


Archivo copiado de D:\DEVELOPMENT\Positron_scripts\Consolidado.parquet a D:\DEVELOPMENT\django_apps\DashSimMov\DashSim\Data\Consolidado.parquet.


In [136]:
import shutil
import os

# Definir las rutas de origen y destino
ruta_origen = r'D:\DEVELOPMENT\Positron_scripts\carga\resultados_vuelos.csv'
ruta_destino = r'D:\DEVELOPMENT\django_apps\DashSimMov\DashSim\Data\resultados_vuelos.csv'

# Verificar si el archivo de origen existe
if os.path.exists(ruta_origen):
    try:
        # Copiar el archivo
        shutil.copy(ruta_origen, ruta_destino)
        print(f'Archivo copiado de {ruta_origen} a {ruta_destino}.')
    except Exception as e:
        print(f'Ocurrió un error al copiar el archivo: {e}')
else:
    print(f'El archivo {ruta_origen} no existe.')

Archivo copiado de D:\DEVELOPMENT\Positron_scripts\carga\resultados_vuelos.csv a D:\DEVELOPMENT\django_apps\DashSimMov\DashSim\Data\resultados_vuelos.csv.


In [137]:
import paramiko

# Configuración del servidor SFTP
hostname = '172.27.251.20'
port = 22
username = 'root'
password = 'daneolx'  # O utiliza una clave privada

# Ruta local y remota
local_file = 'Consolidado.parquet'
remote_file = '/opt/Django/DashSim/DashSim/Data/Consolidado.parquet'

# Crear una sesión SFTP
try:
    # Crear el cliente SSH
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    # Conectar al servidor
    client.connect(hostname, port, username, password)
    
    # Crear la sesión SFTP
    sftp = client.open_sftp()
    
    # Subir el archivo
    sftp.put(local_file, remote_file)
    print(f'Archivo {local_file} subido a {remote_file} en el servidor SFTP.')

except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    # Cerrar la sesión SFTP y el cliente
    if sftp:
        sftp.close()
    if client:
        client.close()

Archivo Consolidado.parquet subido a /opt/Django/DashSim/DashSim/Data/Consolidado.parquet en el servidor SFTP.


In [138]:
import paramiko

# Ruta local y remota
local_file = 'carga/resultados_vuelos.csv'
remote_file = '/opt/Django/DashSim/DashSim/Data/resultados_vuelos.csv'

# Crear una sesión SFTP
try:
    # Crear el cliente SSH
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    # Conectar al servidor
    client.connect(hostname, port, username, password)
    
    # Crear la sesión SFTP
    sftp = client.open_sftp()
    
    # Subir el archivo
    sftp.put(local_file, remote_file)
    print(f'Archivo {local_file} subido a {remote_file} en el servidor SFTP.')

except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    # Cerrar la sesión SFTP y el cliente
    if sftp:
        sftp.close()
    if client:
        client.close()

Archivo carga/resultados_vuelos.csv subido a /opt/Django/DashSim/DashSim/Data/resultados_vuelos.csv en el servidor SFTP.
